In [6]:
# install cpm and import rural two lanes and rural multi lane prediction models for segments
import pip
pip.main(['install','cpm'])
import cpm
from cpm.hsm import rtl_seg,rml_seg

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Defaulting to user installation because normal site-packages is not writeable

Collecting cpm

Using cached cpm-0.8.1.dev71-py3-none-any.whl (25 kB)

Collecting docopt (from cpm)

...

Requirement already satisfied: idna<4,>=2.5 in c:\program files\arcgis\pro\bin\python\envs\arcgispro-py3\lib\site-packages (from requests->cpm) (3.4)

Requirement already satisfied: urllib3<3,>=1.21.1 in c:\program files\arcgis\pro\bin\python\envs\arcgispro-py3\lib\site-packages (from requests->cpm) (1.26.16)

Requirement already satisfied: certifi>=2017.4.17 in c:\program files\arcgis\pro\bin\python\envs\arcgispro-py3\lib\site-packages (from requests->cpm) (2023.7.22)

Installing collected packages: docopt, cpm

Successfully installed cpm-0.8.1.dev69 docopt-0.6.2

In [26]:
rtl_seg.how()

HOW-TO
------
To perform a prediction, call the model's predict method and pass all required independent variable parameters according to the constraints below.

MODEL PARAMETERS
----------------
aadt
- range=[1 to 17800], enforce=snap
ase
- values={0, 1}, enforce=strict
  - Automated speed enforcement
  - 0: not present; 1: present
curve_length
- range=[0 to 100.0], enforce=strict
  - Length of the horizontal curve in miles
curve_radius
- range=[0 to 100000.0], enforce=strict
  - Radius of the horizontal curve in feet
dwy_density
- range=[0 to 100], enforce=snap
  - Number of driveways per mile
grade
- range=[-20 to 20], enforce=strict
lane_width
- range=[6 to 24], enforce=strict
length
- range=[0 to 100.0], enforce=strict
lighting
- values={0, 1}, enforce=strict
  - 0: not present; 1: present
num_years
- range=[0.0 to 100.0], dtype=float, enforce=strict
obs_kabco
- range=[-1 to 1000.0], enforce=strict
  - If historic crash data is unavailable, enter -1 to skip EB analysis.
- obs_year

In [27]:
rml_seg.how()

HOW-TO
------
To perform a prediction, call the model's predict method and pass all required independent variable parameters according to the constraints below.

MODEL PARAMETERS
----------------
aadt
- range=[1 to 89300], enforce=snap
  - where factype: values={4d}
aadt
- range=[1 to 33200], enforce=snap
  - where factype: values={4u}
ase
- values={0, 1}, enforce=strict
  - Automated speed enforcement
  - 0: not present; 1: present
factype
- values={4d, 4u}, enforce=strict
  - 4d: 4-lane divided
  - 4u: 4-lane undivided
lane_width
- range=[6 to 18], enforce=strict
length
- range=[0 to 100.0], enforce=strict
lighting
- values={0, 1}, enforce=strict
  - 0: not present; 1: present
median_width
- range=[0 to 100], enforce=strict
num_years
- range=[0.0 to 100.0], dtype=float, enforce=strict
obs_kabco
- range=[-1 to 1000.0], enforce=strict
  - If historic crash data is unavailable, enter -1 to skip EB analysis.
shld_type
- values={gravel, turf, paved, composite}, enforce=strict
shld_width
-

In [29]:
# loading the sample data
import pandas as pd
df = pd.read_csv('Sample.csv')
print(df.shape,list(df))

(2466, 25) ['factype', 'id', 'length', 'aadt', 'num_years', 'obs_kabco', 'obs_years', 'cf_total', 'lane_width', 'shld_type', 'shld_width', 'curve_radius', 'curve_length', 'spiral_transition', 'se_var', 'rhr', 'rumble_cl', 'twltl', 'ase', 'passing_lanes', 'lighting', 'dwy_density', 'grade', 'median_width', 'sideslope']


In [31]:
gdf = df.groupby('factype').agg({'length':'sum','aadt':'mean','obs_kabco':'sum'})
gdf['KABCOperMile'] = gdf.obs_kabco/gdf.length
gdf

,length,aadt,obs_kabco,KABCOperMile
factype,,,,
4d,498.85,12231.683871,11197,22.445625
4u,11.43,6577.040000,304,26.596675
rtl_seg,467.28,3157.037685,4618,9.882726


In [33]:
import numpy as np
m1 = df.factype.isin(['rtl_seg'])
m2 = df.factype.isin(['4d','4u'])
res1 = rtl_seg.predict(df[m1])
res2 = rml_seg.predict(df[m2])
results = pd.concat([res1,res2])

print(results.shape,list(results))

(2466, 46) ['factype', 'id', 'length', 'aadt', 'num_years', 'obs_kabco', 'obs_years', 'cf_total', 'lane_width', 'shld_type', 'shld_width', 'curve_radius', 'curve_length', 'spiral_transition', 'se_var', 'rhr', 'rumble_cl', 'twltl', 'ase', 'passing_lanes', 'lighting', 'dwy_density', 'grade', 'median_width', 'sideslope', 'spf_kabco', 'af_lane_width', 'af_shld', 'af_hor_curve', 'af_se_var', 'af_grade', 'af_dwy_density', 'af_rumble_cl', 'af_passing_lanes', 'af_twltl', 'af_rhr', 'af_lighting', 'af_ase', 'af_total', 'pred_kabco', 'exp_kabco', 'spf_kabc', 'spf_kab', 'spf_o', 'af_sideslope', 'af_median_width']


In [34]:
results.to_csv('Results.csv',index=False)